In [1]:
import sys
sys.path.append('/app')


In [2]:
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any, Generator


from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

def submit_token(token: str) -> Dict:
    return requests.post(f"{BASE_URL}/submit_token", json={"token": token}).json()

def chat(query: str, thread_id: Optional[str] = None, model: str = "gpt-4o-mini", temperature: float = 0.7) -> Dict:
    response = requests.post(f"{BASE_URL}/chat", headers=headers, json={
        "query": query, "thread_id": thread_id, "model": model, "temperature": temperature
    }, stream=True)
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}
def global_search(query: str, limit: int = 200, min_score: float = 0.4) -> Dict:
    return requests.post(f"{BASE_URL}/search/global", headers=headers, 
        json={"query": query, "limit": limit, "min_score": min_score}).json()

def search_transcripts(query: str, meeting_ids: Optional[List[str]] = None, min_score: float = 0.8) -> Dict:
    return requests.post(f"{BASE_URL}/search/transcripts", headers=headers,
        json={"query": query, "meeting_ids": meeting_ids, "min_score": min_score}).json()

def get_meetings(offset: int = 0, limit: int = 50) -> Dict:
    return requests.get(f"{BASE_URL}/meetings/all", headers=headers, 
        params={"offset": offset, "limit": limit}).json()

def get_meeting_details(meeting_id: str) -> Dict:
    return requests.get(f"{BASE_URL}/meeting/{meeting_id}/details", headers=headers).json()

def create_share_link(target_email: str, access_level: str = "READ", 
                     expiration_hours: int = 24, include_existing: bool = True) -> Dict:
    return requests.post(f"{BASE_URL}/share-links", headers=headers, json={
        "access_level": access_level, "target_email": target_email,
        "expiration_hours": expiration_hours, "include_existing_meetings": include_existing
    }).json()

def accept_share_link(token: str, accepting_email: str) -> Dict:
    return requests.post(f"{BASE_URL}/share-links/accept", headers=headers,
        json={"token": token, "accepting_email": accepting_email}).json()

def start_indexing(num_meetings: int = 200) -> Dict:
    return requests.post(f"{BASE_URL}/start_indexing", headers=headers,
        json={"num_meetings": num_meetings}).json()

def get_indexing_status() -> Dict:
    return requests.get(f"{BASE_URL}/indexing_status", headers=headers).json()

# Add these functions to the initialization cell with other API functions
def get_threads(meeting_id: Optional[str] = None) -> Dict:
    """Get threads for global or meeting-specific context"""
    if meeting_id:
        return requests.get(f"{BASE_URL}/threads/{meeting_id}", headers=headers).json()
    return requests.get(f"{BASE_URL}/threads", headers=headers).json()

def delete_thread(thread_id: str) -> Dict:
    return requests.delete(f"{BASE_URL}/thread/{thread_id}", headers=headers).json()

def chat_meeting(
    query: str, 
    meeting_ids: List[str], 
    thread_id: Optional[str] = None, 
    model: str = "gpt-4o-mini", 
    temperature: float = 0.7
) -> Dict:
    response = requests.post(
        f"{BASE_URL}/chat/meeting", 
        headers=headers, 
        json={
            "query": query,
            "meeting_ids": meeting_ids,
            "thread_id": thread_id,
            "model": model,
            "temperature": temperature
        },
        stream=True
    )
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}

def submit_token(token: str) -> Dict:
    response = requests.post(f"{BASE_URL}/submit_token", json={"token": token})
    result = response.json()
    print(f"User ID: {result.get('user_id')}")
    print(f"User Name: {result.get('user_name')}")
    print(f"Image URL: {result.get('image')}")
    return result

# Example usage:
# r = submit_token(vexa.token)

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [3]:
from vexa import VexaAuth
auth = VexaAuth()

In [5]:
from datetime import timedelta

In [30]:
after_time = datetime.now() - timedelta(seconds=3600)

In [32]:
await auth.get_speech_stats(after_time)

[['b7307556-45d8-424a-84f2-c29ecf6f112c',
  '2024-11-19T12:02:50Z',
  'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
  '2024-11-19T12:06:38Z',
  ['Dmitriy Grankin']],
 ['8c28f07b-cc4c-4660-ab9a-738fe8863999',
  '2024-11-19T11:45:11Z',
  'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
  '2024-11-19T12:01:45Z',
  ['Dmitriy Grankin']],
 ['f74d20fc-d0d5-4835-99fa-23f275c1bd71',
  '2024-11-19T11:50:44Z',
  'fecb31b5-e6b3-4ccd-a275-a25007988aad',
  '2024-11-19T11:57:30Z',
  ['Ahmed Abdelaziz', 'Mark Hurd-Bennett']],
 ['3430c7c3-8753-4f94-8ed8-f3cc26da9f17',
  '2024-11-19T10:59:39Z',
  '8fbdb9bd-a4b3-46c0-8e78-87868d7aef0a',
  '2024-11-19T11:16:29Z',
  ['Beatriz Lerer', 'Lara Vargas']],
 ['2bda516a-07bb-44e1-9626-0332fa95e2ee',
  '2024-10-22T11:00:25Z',
  'fd57278e-1cdc-4ad9-b12e-837ac39e9a55',
  '2024-11-19T11:09:57Z',
  ['Aleksei C.', 'Alexander C.', 'Anh D.', 'Nick M.', 'Nikita M.']],
 ['311c28f0-0e94-4fcc-891a-c0176f74e6c7',
  '2024-10-11T11:01:27Z',
  'fd57278e-1cdc-4ad9-b12e-837ac39e9a55',
  '2024-11-

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(get_meetings()['meetings'])

In [11]:
df['timestamp'] = pd.to_datetime(df['timestamp'],format='mixed')

In [12]:
df.sort_values(by='timestamp', ascending=False)


,meeting_id,meeting_name,timestamp,is_indexed,speakers
0,2aeecf38-313d-490a-b07d-aa95c95ee8ea,09:43,2024-11-19 09:43:12.000000,False,[Dmitriy Grankin]
1,7afff477-c0bd-44c1-92c9-75f94ac75d68,20:14,2024-11-18 20:00:12.930000,False,"[Dmitriy Grankin, Sergey Ryabenko]"
2,582f4e41-572c-47e9-8feb-0f485f31728b,12:27,2024-11-18 12:19:28.188000,False,[Dmitriy Grankin]
3,dc5e75bd-392c-4185-b842-d5164578dcdc,12:03,2024-11-18 12:03:22.000000,False,[Dmitriy Grankin]
4,a3e21b99-7b2f-4d19-b2e4-8a2d8749dcd7,10:30,2024-11-17 10:30:26.000000,False,[Dmitriy Grankin]
5,7973854d-ce5a-4263-b0f5-74007ea7a630,15:15,2024-11-15 14:32:30.220000,False,"[Dmitriy Grankin, Olga Nemirovskaya]"
6,ce84fa05-fe62-4484-8114-a1af9eed60e3,14:21,2024-11-15 14:21:43.000000,False,[Dmitriy Grankin]
7,140b3e35-1fc9-4905-abb3-e608243a8543,14:12,2024-11-15 14:12:21.000000,False,[Dmitriy Grankin]
8,3da3fc7d-0282-4b7f-b3dd-3e39ca9dcf05,12:28,2024-11-15 12:28:52.000000,False,[Dmitriy Grankin]
9,76065277-085c-4c86-bc95-5ca0e10a8147,11:58,2024-11-15 11:58:05.000000,False,[Dmitriy Grankin]


In [7]:
get_meeting_details('26045454-6226-4ecb-bfc4-46a3a1e219eb')

{'meeting_id': '26045454-6226-4ecb-bfc4-46a3a1e219eb',
 'timestamp': '2024-07-23T09:17:26.198151',
 'meeting_name': '10:04',
 'transcript': '[{"speaker": "Inferno Maverick", "speaker_id": "TBD", "content": " attenzione veloce", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:17:26.198151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Good morning, everybody\'s here, I\'m Andre, you\'ll be missing, how are you today?", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:17:59.824151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Good, but it\'s a little bit hard to hear you.", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:18:05.584151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Yeah, any better?", "html_content": null, "html_content_short": null, "keywords": [], "time